# Network isochrones

### Original code, obtained from:
https://github.com/gboeing/osmnx-examples/blob/main/notebooks/13-isolines-isochrones.ipynb

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# configure the place, network type, trip times, and travel speed
network_type = "walk"
trip_times = [10]  # in minutes
travel_speed = 5  # walking speed in km/hour

epsg = single_point.crs.to_epsg()
transformer = pyproj.Transformer.from_crs(f"epsg:{epsg}", "epsg:4326")
place = transformer.transform(single_point['geometry'].x, single_point['geometry'].y)

G = ox.graph_from_point(place, network_type=network_type, dist=1000)

# find the centermost node and then project the graph to UTM
gdf_nodes = ox.graph_to_gdfs(G, edges=False)
x, y = gdf_nodes["geometry"].unary_union.centroid.xy
center_node = ox.distance.nearest_nodes(G, x[0], y[0])
G = ox.project_graph(G)
graph_epsg = G.graph['crs'].to_epsg()

# add an edge attribute for time in minutes required to traverse each edge
meters_per_minute = travel_speed * 1000 / 60  # km per hour to m per minute
for _, _, _, data in G.edges(data=True, keys=True):
    data["time"] = data["length"] / meters_per_minute

# get one color for each isochrone
iso_colors = ox.plot.get_colors(n=len(trip_times), cmap="plasma", start=0, return_hex=True)

def make_iso_polys(G, edge_buff=25, node_buff=50, infill=False):
    isochrone_polys = []
    for trip_time in sorted(trip_times, reverse=True):
        subgraph = nx.ego_graph(G, center_node, radius=trip_time, distance="time")

        node_points = [sg.Point((data["x"], data["y"])) for node, data in subgraph.nodes(data=True)]
        nodes_gdf = gpd.GeoDataFrame({"id": list(subgraph.nodes)}, geometry=node_points)
        nodes_gdf = nodes_gdf.set_index("id")

        edge_lines = []
        for n_fr, n_to in subgraph.edges():
            f = nodes_gdf.loc[n_fr].geometry
            t = nodes_gdf.loc[n_to].geometry
            edge_lookup = G.get_edge_data(n_fr, n_to)[0].get("geometry", sg.LineString([f, t]))
            edge_lines.append(edge_lookup)

        n = nodes_gdf.buffer(node_buff).geometry
        e = gpd.GeoSeries(edge_lines).buffer(edge_buff).geometry
        all_gs = list(n) + list(e)
        new_iso = gpd.GeoSeries(all_gs).unary_union

        # try to fill in surrounded areas so shapes will appear solid and
        # blocks without white space inside them
        if infill:
            new_iso = sg.Polygon(new_iso.exterior)
        isochrone_polys.append(new_iso)
    return isochrone_polys


# make the isochrone polygons
isochrone_polys = make_iso_polys(G, edge_buff=25, node_buff=0, infill=True)
gdf = gpd.GeoDataFrame(geometry=isochrone_polys, crs=f"EPSG:{graph_epsg}")

# plot the network then add isochrones as colored polygon patches
fig, ax = ox.plot_graph(
    G, show=False, close=False, edge_color="#999999", edge_alpha=0.2, node_size=0
)
gdf.plot(ax=ax, color=iso_colors, ec="none", alpha=0.6, zorder=-1)
plt.show()


### Modified code

In [ ]:
# configure the place, network type, trip times, and travel speed
network_type = "walk"
trip_time = 10  # in minutes
travel_speed = 4.5  # walking speed in km/hour

epsg = single_point.crs.to_epsg()
transformer = pyproj.Transformer.from_crs(f"epsg:{epsg}", "epsg:4326")
place = transformer.transform(single_point['geometry'].x, single_point['geometry'].y)

G = ox.graph_from_point(place, network_type=network_type, dist=1000)

# find the centermost node and then project the graph to UTM
gdf_nodes = ox.graph_to_gdfs(G, edges=False)
x, y = gdf_nodes["geometry"].unary_union.centroid.xy
center_node = ox.distance.nearest_nodes(G, x[0], y[0])
G = ox.project_graph(G)
graph_epsg = G.graph['crs'].to_epsg()

# add an edge attribute for time in minutes required to traverse each edge
meters_per_minute = travel_speed * 1000 / 60  # km per hour to m per minute
for _, _, _, data in G.edges(data=True, keys=True):
    data["time"] = data["length"] / meters_per_minute

def make_iso_poly(G, edge_buff=25, node_buff=0):
    subgraph = nx.ego_graph(G, center_node, radius=trip_time, distance="time")

    node_points = [sg.Point((data["x"], data["y"])) for node, data in subgraph.nodes(data=True)]
    nodes_gdf = gpd.GeoDataFrame({"id": list(subgraph.nodes)}, geometry=node_points)
    nodes_gdf = nodes_gdf.set_index("id")

    edge_lines = []
    for n_fr, n_to in subgraph.edges():
        f = nodes_gdf.loc[n_fr].geometry
        t = nodes_gdf.loc[n_to].geometry
        edge_lookup = G.get_edge_data(n_fr, n_to)[0].get("geometry", sg.LineString([f, t]))
        edge_lines.append(edge_lookup)

    n = nodes_gdf.buffer(node_buff).geometry
    e = gpd.GeoSeries(edge_lines).buffer(edge_buff).geometry
    all_gs = list(n) + list(e)
    isochrone_poly = gpd.GeoSeries(all_gs).unary_union

    # try to fill in surrounded areas so shapes will appear solid and
    # blocks without white space inside them
    isochrone_poly = sg.Polygon(isochrone_poly.exterior)
    
    return isochrone_poly


# make the isochrone polygons
isochrone_poly = make_iso_poly(G)
gdf = gpd.GeoDataFrame(geometry=[isochrone_poly], crs=f"EPSG:{graph_epsg}")

# get one color for each isochrone
iso_colors = ox.plot.get_colors(n=1, cmap="plasma", start=0, return_hex=True)

# plot the network then add isochrones as colored polygon patches
fig, ax = ox.plot_graph(
    G, show=False, close=False, edge_color="#999999", edge_alpha=0.2, node_size=0
)
gdf.plot(ax=ax, color=iso_colors, ec="none", alpha=0.6, zorder=-1)
plt.show()
